In [1]:
import pandas as pd
import numpy as np

In [2]:
data_col=['user_id','shop_id','time_stamp']

In [3]:
#data_view=pd.read_csv('../dataset/user_view.txt',header=None)

#data_view.columns=data_col

In [4]:
data_pay=pd.read_csv('../dataset/user_pay.txt',header=None)

data_pay.columns=data_col

In [5]:
data_pay.head()

,user_id,shop_id,time_stamp
0,22127870,1862,2015-12-25 17:00:00
1,3434231,1862,2016-10-05 11:00:00
2,16955285,1862,2016-02-10 15:00:00
3,13799128,1862,2016-01-13 14:00:00
4,13799128,1862,2016-07-05 12:00:00


In [6]:
data_pay.time_stamp=pd.to_datetime(data_pay.time_stamp, format='%Y/%m/%d %H:%M:%S')

In [7]:
data_pay['data']=data_pay.time_stamp.dt.date

In [8]:
data_pay.head()

,user_id,shop_id,time_stamp,data
0,22127870,1862,2015-12-25 17:00:00,2015-12-25
1,3434231,1862,2016-10-05 11:00:00,2016-10-05
2,16955285,1862,2016-02-10 15:00:00,2016-02-10
3,13799128,1862,2016-01-13 14:00:00,2016-01-13
4,13799128,1862,2016-07-05 12:00:00,2016-07-05


In [9]:
feature_daypay=pd.DataFrame(data_pay.groupby(['shop_id','data'])['user_id'].count())

In [10]:
feature_daypay.rename(columns={'user_id':'daypay'},inplace=True)

In [11]:
feature_daypay.to_csv('../tempfile/feature_daypay.csv')